Experiment 1: Yeilds the same training policy when the cross entropies are weighted against each other 

In [10]:

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tensorflow
sess = tf.InteractiveSession()

def weight_variable(shape):
	initial = tf.truncated_normal(shape, stddev=0.1)
	return tf.Variable(initial)

def bias_variable(shape):
	initial = tf.constant(.1, shape=shape)
	return tf.Variable(initial)

def conv2d(x, W):
	return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
	return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
							strides = [1, 2, 2, 1], padding='SAME')


x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
y_expert = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])


probs = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


temperature = 10
t_probs = tf.nn.softmax(tf.div(probs, temperature))
y_conv = tf.nn.softmax(probs)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
cross_entropy_expert = tf.reduce_mean(-tf.reduce_sum(y_expert * tf.log(t_probs), reduction_indices=[1])) 
train_step = tf.train.AdamOptimizer(1e-4).minimize(.2 * cross_entropy + .8 * cross_entropy_expert )
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(2000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], y_expert: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))

  train_step.run(feed_dict={x: batch[0], y_: batch[1], y_expert: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.06
step 100, training accuracy 0.8
step 200, training accuracy 0.9
step 300, training accuracy 0.06
step 400, training accuracy 0.08
step 500, training accuracy 0.08
step 600, training accuracy 0.06
step 700, training accuracy 0.14
step 800, training accuracy 0.06
step 900, training accuracy 0.1
step 1000, training accuracy 0.14
step 1100, training accuracy 0.08
step 1200, training accuracy 0.1
step 1300, training accuracy 0.08
step 1400, training accuracy 0.12
step 1500, training accuracy 0.06
step 1600, training accuracy 0.06
step 1700, training accuracy 0.08
step 1800, training accuracy 0.1
step 1900, training accuracy 0.12
test accuracy 0.098


In [1]:

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tensorflow
sess = tf.InteractiveSession()

def replace_none_with_zero(l):
    return [0 if i==None else i for i in l]

def weight_variable(shape):
	initial = tf.truncated_normal(shape, stddev=0.1)
	return tf.Variable(initial)

def bias_variable(shape):
	initial = tf.constant(.1, shape=shape)
	return tf.Variable(initial)

def conv2d(x, W):
	return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
	return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
							strides = [1, 2, 2, 1], padding='SAME')


x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
y_expert = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])


probs = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


temperature = 10.0
temp_square = temperature * temperature 
t_probs = tf.nn.softmax(tf.div(probs, temperature))
y_conv = tf.nn.softmax(probs)


cross_entropy = .9* tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
cross_entropy_expert = .1 * tf.reduce_mean(-tf.reduce_sum(y_expert * tf.log(t_probs), reduction_indices=[1])) 
optim = tf.train.AdamOptimizer(1e-4)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




grads_and_vars = optim.compute_gradients(cross_entropy)


temp_grads = replace_none_with_zero(optim.compute_gradients(cross_entropy_expert))

temp_compensated_grads = [(grad * temp_square, var) for grad, var in temp_grads]



#train_op = optim.apply_gradients(grads_and_vars + grads_and_vars_two)

train_op = optim.apply_gradients(temp_compensated_grads + grads_and_vars)

#grads_and_vars_soft = optim.compute_gradients(cross_entropy_expert)

#grads_and_vars_hard = optim.compute_gradients(cross_entropy)

sess.run(tf.initialize_all_variables()) 


#sess.run(optim.apply_gradients(grads_and_vars_hard + grads_and_vars_soft))




for i in range(2000):

  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], y_expert: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))

  train_op.run(feed_dict={x: batch[0], y_: batch[1], y_expert: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.1
step 300, training accuracy 0.06
step 400, training accuracy 0.08
step 500, training accuracy 0.08
step 600, training accuracy 0.06
step 700, training accuracy 0.14
step 800, training accuracy 0.06
step 900, training accuracy 0.1
step 1000, training accuracy 0.14
step 1100, training accuracy 0.1
step 1200, training accuracy 0.08
step 1300, training accuracy 0.1
step 1400, training accuracy 0.12
step 1500, training accuracy 0.12
step 1600, training accuracy 0.04
step 1700, training accuracy 0.08
step 1800, training accuracy 0.06
step 1900, training accuracy 0.08
test accuracy 0.098
